In [1]:
import numpy as np
import pandas as pd
from CODECbreakCode.AudioMixer import FullTrackAudioMixer
import CODECbreakCode.Evaluator as Evaluator
from CODECbreakCode.Evaluator import MeasureHAAQIOutput
import argparse
from Optimiser.config import get_config, normalize_action, denormalize_action

In [2]:
import warnings
warnings.filterwarnings("ignore", "Possible clipped samples in output.")
warnings.filterwarnings("ignore",message="Warning: input samples dtype is np.float64. Converting to np.float32")

In [3]:
Reggae_Mixing_Path = '/home/codecrack/CodecBreakerwithRL/AudioEX/Reggae'
Reggae_Noise_Generator_MP3 = FullTrackAudioMixer(Reggae_Mixing_Path)
#Noise_Generator_MP3.ManipulateInitGAIN([0, 0, 0, 0])
Reggae_Referece_File = Reggae_Noise_Generator_MP3.TestDynNoisedFullTrack([0]*24,"Reference_IN_FULL.wav",isNormalised=False,isCompensated=True)
print(f"Referece_File:{Reggae_Referece_File}")

Reggae_Referece_MP3File = Evaluator.Mp3LameLossyCompress(Reggae_Referece_File,64)
print(f"Referece_MP3File:{Reggae_Referece_MP3File}")

Vocal duration orginal is 17.462666666666667 seconds, now is the 8.0, the audio changing to the MONO
Drum duration orginal is 17.462666666666667 seconds, now is the 8.0, the audio changing to the MONO
Bass duration orginal is 17.462666666666667 seconds, now is the 8.0, the audio changing to the MONO
Other duration orginal is 17.462666666666667 seconds, now is the 8.0,  the audio changing to the MONO
Mixing File Load Sucessful
Referece_File:/home/codecrack/CodecBreakerwithRL/AudioEX/Reggae/Mixing_Result/Reference_IN_FULL.wav
Referece_MP3File:/home/codecrack/CodecBreakerwithRL/AudioEX/Reggae/Mixing_Result_Mp3_Wav/Reference_IN_FULL_64kbps.wav


In [4]:
####initalise the Haaqi
MeasureHAAQI = MeasureHAAQIOutput(Reggae_Referece_MP3File)#Initilize the HAAQI with a permanent reference
MeasureHAAQI.MeasureHAQQIOutput(Reggae_Referece_MP3File) #Test on how far from itself to itself

0.9952523968235077

In [5]:
import tempfile
import os
BASE_DIR = "/home/codecrack/CodecBreakerwithRL/AudioEX/Reggae/Mixing_Result"
TMP_SUBDIR = os.path.join(BASE_DIR, "tmp")

# make sure it exists once at startup:
os.makedirs(TMP_SUBDIR, exist_ok=True)
def haaqi_reward_muti_fn(solution: np.ndarray, is_normalised=True) -> float:
    if is_normalised:
        solution = denormalize_action(solution)
#    print(f'solution:{solution}, the type is {type(solution), type(solution[0])}')
 
    # Create a unique temp‐file name
    fd, degradated_filename = tempfile.mkstemp(prefix="dynC_", suffix=".wav")
    os.close(fd)  # we’ll let your compressor write to that path

    try:
        solution = list(solution)
        gener_Audio = Reggae_Noise_Generator_MP3.TestDynNoisedFullTrack(solution,degradated_filename,isNormalised=False,isCompensated=True)
        gener_Audio_mp3 = Evaluator.Mp3LameLossyCompress(gener_Audio, 64)
        score = MeasureHAAQI.MeasureHAQQIOutput(gener_Audio_mp3)
    finally:
        # clean up
        try:
            os.remove(degradated_filename)
        except OSError:
            pass

    return round(1 - score, 4)

### PPO Trainning

In [6]:
import Optimiser

In [7]:
#import Optimiser.continous_RL_train_PPO
from Optimiser.continous_RL_train_PPO import continous_RL_train_PPO as CRLTrain

2025-07-30 02:31:22.832581: I tensorflow/core/util/port.cc:113] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2025-07-30 02:31:22.859273: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:9261] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2025-07-30 02:31:22.859302: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:607] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2025-07-30 02:31:22.861147: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1515] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2025-07-30 02:31:22.869765: I tensorflow/core/platform/cpu_feature_guar

In [8]:
trainner = CRLTrain(sub_episode_length=3, sub_episode_num_single_batch=3, env_num=1)
trainner.set_environments(haaqi_reward_muti_fn)
trainner.train(update_num=1, eval_intv=2)

number of sub_episodes used for a single param update: 3
train_env.batch_size = parallel environment number =  1
update_num: 1, eval_intv: 2
Instructions for updating:
Use `as_dataset(..., single_deterministic_pass=True)` instead.
Epoch 0: policy_loss=-0.104  value_loss=78.083  entropy_loss=-0.022  clip_fraction=0.333
final reward before udpate: -1000000000.0
final reward after udpate: 0.3677
updated final_solution= [ 60.     2.76 -22.65   1.     1.   283.77  35.     3.     0.     2.12
   1.   260.77  35.     3.   -25.     4.77   1.   200.    60.     3.
 -12.45   5.     1.   200.  ]
train_step no.= 0
best_solution of this generation= [ 60.     2.76 -22.65   1.     1.   283.77  35.     3.     0.     2.12
   1.   260.77  35.     3.   -25.     4.77   1.   200.    60.     3.
 -12.45   5.     1.   200.  ]
best step reward= 0.3677
avg step reward= 0.31701666
obs_mean: [ 60.     2.19 -14.76   2.15   4.27 260.82  43.65   2.39 -13.53   3.33
   4.34 350.62  44.63   2.76 -22.97   3.06   6.98 383.

In [ ]:
solution =[60.0, 2.15, -17.61, 1, 1, 418.71, 47.33, 3, -5.16, 4.17, 2.41, 286.74, 57.16, 0.56, -24.19, 1.25, 3.92, 317.3, 51.2, 2.29, -10.61, 5, 2.01, 361.39]
haaqi_reward_muti_fn(solution, is_normalised=False)

In [ ]:
print(type(trainner._REINFORCE_agent.policy))            # GreedyPolicy
print(type(trainner._REINFORCE_agent.policy.wrapped_policy))  # Base TFPolicy (e.g., ActorPolicy)

In [9]:
Reggae_Noise_Generator_MP3.EraseTheMp3Mixing()

Cleared all contents in '/home/codecrack/CodecBreakerwithRL/AudioEX/Reggae/Mixing_Result'.
Cleared all contents in '/home/codecrack/CodecBreakerwithRL/AudioEX/Reggae/Mixing_Result_Mp3'.
Cleared all contents in '/home/codecrack/CodecBreakerwithRL/AudioEX/Reggae/Mixing_Result_Mp3_Wav'.


In [10]:
from tf_agents.policies import policy_saver

saver = policy_saver.PolicySaver(trainner._REINFORCE_agent.policy)        
#saver.save(policy_dir)

TypeError: this __dict__ descriptor does not support '_DictWrapper' objects

In [ ]:
def save_policy(filefold):
        if filefold is None:
            raise ValueError("filefold cannot be None")
        else:
            if not os.path.exists(filefold+ 'Policy/'): 
                os.makedirs(filefold+ 'Policy/')

        policy_dir = os.path.join(filefold,'Policy/', 'exported_policy')
        saver = policy_saver.PolicySaver(self._REINFORCE_agent.policy)
        saver.save(policy_dir)